<a href="https://colab.research.google.com/github/JessicaArauj/AI_Agent_with_Gemini/blob/main/AI_Agent_with_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

# Imports

In [2]:
from typing import Dict, List, Literal

from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

# Variables

In [3]:
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

# Defining model and temperature

In [4]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1,
    api_key = GOOGLE_API_KEY
)

# Chat test

In [12]:
resp_test = llm.invoke("Who are you in detail?")
#print(resp_test)
print(resp_test.content)

I am a **large language model**, a type of artificial intelligence. Here's a more detailed breakdown:

1.  **What I Am:**
    *   **An AI/Computer Program:** I am not a human, a physical being, or a sentient entity. I don't have a body, consciousness, emotions, personal experiences, or opinions.
    *   **A Large Language Model (LLM):** This means I'm a complex neural network specifically designed to understand, process, and generate human-like text. I've been trained on an enormous dataset of text and code.
    *   **Developed by Google:** My underlying architecture and training were created and refined by Google.

2.  **How I Work (Simplified):**
    *   **Pattern Recognition:** I learn patterns, grammar, facts, and relationships within the vast amount of data I've been trained on.
    *   **Prediction:** When you give me a prompt, I analyze the input and predict the most probable and relevant sequence of words to generate a coherent and helpful response, based on the patterns I've l

# Prompt screening

In [13]:
screening_prompt = (
    "You are a service desk triage agent for internal policies at JTA Development. Given the users message, return ONLY a JSON with: \n"
    "{\n"
    " \"decision\": \"HIGH_SCALABLE\" | \"REQUEST_INFORMATION\" | \"OPEN_TICKET\", \n"
    " \"urgency\": \"LOW\" | \"MEDIUM\" | \"HIGH\", \n"
    " \"missing_fields\": (\"...\"),\n"
    " }\n"
    "Rules \n"
    " - \"HIGH_SCALABLE\": Objective and clear questions about rules or procedures described in the policies (Ex: \"I need a refund for my home office internet\"), (\"How does the food policy work when traveling?\") \n"
    " - \"REQUEST_INFORMATION\": User message that is vague or lacks information to identify the topic or context (Ex: \"I need help with a policy\"), (\"I have a general question\") \n"
    " - \"OPEN_TICKET\": Requests for exception, release, approval, or special access. Or, when the user explicitly requests opening a ticket (Ex: \"I want an exception to work remotely for 5 days) \n"
    " Analyze the message and decide on the most appropriate action."
)

In [14]:
class screeningOut(BaseModel):
  decision: Literal["HIGH_SCALABLE", "REQUEST_INFORMATION","OPEN_TICKET"]
  urgency: Literal["LOW", "MEDIUM", "HIGH"]
  missing_fields: List[str] = Field(default_factory=list)

# LLM Screening

In [15]:
llm_screening = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1,
    api_key = GOOGLE_API_KEY
)

# Function

In [21]:
screening_chain = llm_screening.with_structured_output(screeningOut)

def screening(message: str) -> Dict:
  output: screeningOut = screening_chain.invoke([
      SystemMessage(content = screening_prompt),
      HumanMessage(content = message)
  ])

  return output.model_dump()

# Examples

In [22]:
tests = ("Can I get a refund for the internet?",
         "I can work remotely for an additional 10 days. How do I request it?",
         "Can I refund training and courses?"
         )

# Interaction

In [23]:
for message_tests in tests: print(f"Questions: {message_tests} \n -> Response: {screening(message_tests)}\n")

Questions: Can I get a refund for the internet? 
 -> Response: {'decision': 'HIGH_SCALABLE', 'urgency': 'LOW', 'missing_fields': []}

Questions: I can work remotely for an additional 10 days. How do I request it? 
 -> Response: {'decision': 'OPEN_TICKET', 'urgency': 'MEDIUM', 'missing_fields': []}

Questions: Can I refund training and courses? 
 -> Response: {'decision': 'HIGH_SCALABLE', 'urgency': 'LOW', 'missing_fields': []}

